In [142]:
import cv2
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow

%matplotlib inline
import time

In [149]:
def genProposals(image_name):
    s = time.time()
    img = Image.open(image_name)

    h,w = img.size
    base = min(h,w)
    cur_size = 250
    
    props = []
    count = 0
    while cur_size <= base:
        for a in range(0,w,cur_size):
            for b in range(0,h,cur_size):
                crop = img.crop((a,b,a+cur_size,b+cur_size))
                if np.var(np.mean(crop,2)) < 400: continue
                crop = crop.resize((250,250))
                #crop.save("../data/thumb/%d%d%d.jpg"%(a,b,cur_size))
                
                count += 1
                props.append(((a,b,cur_size),crop))
        cur_size *= 2    
        
    print("Generated %d images"%count)
    print(time.time()-s)
    return props

In [150]:
genProposals('../data/small.jpeg')

Generated 232 images
5.818382978439331


[((0, 5000, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A456358>),
 ((250, 5000, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A4566A0>),
 ((500, 4500, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A4562B0>),
 ((500, 4750, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A4500F0>),
 ((500, 5000, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A450160>),
 ((750, 5000, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A450080>),
 ((1000, 5000, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A4501D0>),
 ((1250, 2000, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A450240>),
 ((1250, 2250, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A450320>),
 ((1250, 2500, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A450390>),
 ((1250, 3750, 250),
  <PIL.Image.Image image mode=RGB size=250x250 at 0x7FCF4A450400>),
 ((1250, 4000, 250),
  <PIL.I